# Investigation of planarity of agglomerative clusters

In [1]:
import numpy as np
from scipy.cluster.hierarchy import ward, fcluster

In [ ]:
ground_truth = np.load('../data/synthetics_31.npy')
X = np.concatenate(ground_truth, axis = 1)